#### imports & utility funcs

In [450]:
import re
import os
import pandas as pd
from tqdm.notebook import tqdm

def ld_nh(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
def listdir_nohidden(path):
    return list(ld_nh(path))

data_dir = 'raw'
files = listdir_nohidden(data_dir)

### data source: https://results.enr.clarityelections.com/GA/105369/web.264614/

work with the singular incoming .txt file by manually delimited lines (`'\n'`) and cells (`' '*2`)

In [451]:
def read_data(file):
    
    with open(os.path.join(data_dir, file)) as f:
        f = f.read()
    
    delim = lambda x: re.split('(?:\s){2,}', x) # delimit cells by 3 or more spaces
    rows = [delim(r) for r in f.split('\n')] # delimit rows with line breaks
    
    # function to get the category of each race
    # ( for organizing the repository )
    def parse_cat(race):
        if 'Service' in race:
            return ' '.join(race.split()[:3])
        else: return ' '.join(race.split()[:2]).replace('President of', 'US President')


    # find the consecutive batch of rows associated with each race

    data = [] # to compile info on all races

    for i, row in enumerate(rows[1:-2]):
        
        row_data = {}
        if row==['']: # indicates the beginning of data on a row

            if i>10: # identify the last row of & save the previous race
                last_row_data['data_ends'] = i-1
                last_row_data['data'] = rows[ last_row_data['data_starts'] : i ] 
                data.append(last_row_data)         

            if i < (len(rows)-10): # identify the first row of & other details on this race 
                row_data['race'] = ''.join(rows[i+2])
                row_data['race_cat'] = parse_cat(row_data['race'])
                row_data['candidates'] = rows[i+3]
                row_data['data_starts'] = i+4 
                last_row_data = row_data.copy()
    return data[1:]
    print(len(data), 'total races found.')

#### define desired column suffixes:

In [514]:
stats_should_be = ['Election Day Votes', 'Advanced Voting Votes', 'Absentee by Mail Votes', 'Provisional Votes', 'TOTAL VOTES']

append the candidate names to the corresponding cols describing vote type counts

In [438]:
def rename_cols(cols, candidates):
    # find sum total col
    for i, col in enumerate(cols):
        if col=='Total': final_total_col = i
            
    for i, col in enumerate(cols):
        if 'Total' in col and i!=final_total_col: cols[i] = 'Choice Total'

    new_cols = []
    
    for i, col in enumerate(cols):
        if i == final_total_col:
            new_cols.append(col)
        elif 'County' in col or 'Precinct' in col or 'Voters' in col or len(col)<3:
            new_cols.append(col)
        else:
            new_cols.append(candidates[0]+'_'+col)
            if 'Total' in col:
                candidates = candidates[1:]

    return new_cols           

#### load data for one race, parse with above functions.

In [439]:
def parse_data(data, race_idx, county_name, target_dir='clean_by_county'):
    race_data = pd.DataFrame(data[race_idx]['data'])
    race_data.columns = race_data.loc[0]
    race_data.drop(0, inplace=True)
    
    
    # most counties used "county" instead of "precinct" (all are actually precincts)
    if 'County' in race_data.columns:
        race_data.rename(columns={'County': 'Precinct'}, inplace=True) 
    race_data = race_data.set_index('Precinct')
    race_data['County'] = county_name.replace('_', ' ')

    
    # add candidate names to specific columns
    candidates = [c for c in data[race_idx]['candidates'] if len(c)>1]
    cols = list(race_data.columns)
    
    if False: # len(candidates)>5:
        print(county_name.upper())
        print(candidates)
        print(cols)
        print('\n\n\n')
    
    # run column renaming function, adding candidates to features
        
    new_cols = rename_cols(cols, candidates.copy())
    race_data.columns = new_cols.copy()
        
    # create directories & file details
    race_name = data[race_idx]['race'].split('/')[0]
    
    if target_dir not in os.listdir():
        os.makedirs(f'{target_dir}/')
    if county_name not in os.listdir(target_dir):
        os.makedirs(f'{target_dir}/{county_name}/')

    # save csv
    race_data = race_data[[c for c in race_data.columns if 'Unnamed' not in c]]
    race_data.reset_index(inplace=True)
    race_data = race_data[['County'] + [c for c in race_data.columns if c!='County']]
    race_data.to_csv(f'{target_dir}/{county_name}/{race_name}.csv', index=False)

---

---

---


## parse & save data

In [474]:
for file in tqdm(sorted(files)): # iterate counties
    data = read_data(file)

    for race_idx in range(len(data)): # iterate races within county
        parse_data(data.copy(), race_idx, file.split('.')[0])

# load & join parsed data

In [574]:
clean_dir = 'clean_by_county'
counties = listdir_nohidden(clean_dir)

### compile data for one race

In [575]:
race_hotword = 'US Senate (Perdue)' ## SET THE RACE NAME HERE, then run all cells below.

In [576]:
# id all files for the target race
race_files = []
for county in counties:
    fpath = os.path.join(clean_dir, county)
    county_files = listdir_nohidden(fpath)
    found = False
    for f in county_files:
        if race_hotword in f:
            race_files.append( os.path.join(fpath, f) )
            found = True
    if not found: print('Not found:', county)

# concat data from all counties
merged = pd.concat( [pd.read_csv(f) for f in race_files] )
merged.reset_index(drop=True, inplace=True)

### fix typo
combine votes for lieberman/lie**r**berman

In [577]:
# ADD VOTES FOR MATT lie*R*berman to MATT LIEBERMAN:
def fix_lieberman(merged):
    
    # fill missing good spellings values from bad spellings column 
    good_spell = [c for c in merged.columns if 'Lieber' in c]
    bad_spell = [c for c in merged.columns if 'Lierber' in c]
    merged[good_spell] = merged[good_spell].fillna(merged[bad_spell])
    
    def fillna_info(c):
        n = c.split('(')[0].strip()
        n = n[:10].replace('r', 'R') + n[10:]
        print('Filling', merged[c].isna().sum(), f'nan values for {n}...')
        merged[c] = merged[c].fillna(0)
        return merged

    
    true = 'Matt Lieberman (Dem)_Choice Total'
    typo = 'Matt Lierberman (Dem)_Choice Total'
    merged = fillna_info(true)
    merged = fillna_info(typo)
    merged[true] += merged[typo]

    print(f'+ {int(sum(merged[true])-sum(merged[typo]))} correct votes'.upper())
    print(f'     {int(sum(merged[typo]))} misspelled'.upper())
    print('—'*24)
    print(f'  {int(sum(merged[true]))} total votes'.upper())
    return merged

if 'Loeffler' in race_hotword:
    merged = fix_lieberman(merged)

## Fix horizontal value shift (errors in .txt delimiter) 

**by index**...
 In a few rows, all the values are shifted over to the left because the registered voter number was swallowed by the preceding "precinct" column
 
 So extract the precinct number with regex, then shift all the columns over the right. 
 
 These rows are detectable becaus the "Total" column is empty, and the values are visibly shifted.

In [578]:
for i in merged[merged.Total.isna()].index:
    #print(i, merged.loc[i, 'County'], merged.loc[i, 'Precinct'])
    for feat, val in dict(merged.loc[i]).items():
        if feat=='Precinct':
            ns_found = re.findall('[\d]*', val)
            n_registered = [n for n in ns_found if n!=''][-1]
            #print(n_registered)
        #print(feat, val)
        
    # shift all values over by one.
    merged.iloc[i:i+1, 2:] = merged.iloc[i:i+1, 2:].shift(1, axis=1)
    
    # add re-persed registered number
    merged.iloc[i:i+1, 2] = n_registered

## identify failed rows (by vote total)

#### (check if all candidates' votes add up correctly in each precinct)

#### this loop also fixes things for lieberman in a few areas

In [579]:
def verify_totals(merged, verb=False):
    bad_math = []
    totals = merged[[c for c in merged.columns if 'total' in c.lower()]]
    for ridx in totals.index:
        if str(totals.loc[ridx, 'Total']).lower()=='nan':
            if verb: print('NaN val.:', ridx)
        elif totals.iloc[ridx].sum()/2 != totals.loc[ridx, 'Total']:
            if verb: print('Bad Math:', ridx, '|', totals.iloc[ridx].sum()/2, '!=', totals.loc[ridx, 'Total'])
            bad_math.append(ridx)
    return bad_math

bad_math = verify_totals(merged)

In [580]:
# define candidates in this race
cands = []
for c in merged.columns:
    if ')_' in c:
        cands.append(c.split('_')[0])      
cands = list(set(cands))
if 'Loeff' in race_hotword: cands.remove('Matt Lierberman (Dem)')


# define proper and typo version of MATT LIE(r)BERMAN
# -- to replace values in the correct spelling column when necessary
good_spell = [c for c in merged.columns if 'Lieber' in c]
bad_spell = [c for c in merged.columns if 'Lierber' in c]


for i in tqdm(merged.index): # iterate precincts
    for cand in cands: # iterate all candidates in this race
        cand_cols = [c for c in merged.columns if cand in c]
        
        disag_cols = cand_cols[:-1] # names of cols of vote types
        cand_t_col = cand_cols[-1] # name of col for total votes
        
        total_init = merged.iloc[i:i+1][cand_t_col].values[0]
        sub_counts = merged.iloc[i:i+1][disag_cols]
        
        # if values are missing for the correct spelling of Lieberman...
        if i in bad_math:
            # just use the mispelled values directly (instead of the calced, added version)
            merged.loc[i, good_spell] = merged.loc[i, bad_spell].values
            
        sub_counts = merged.iloc[i:i+1][disag_cols]
        total_calc = sub_counts.values.sum()
        
        if total_init != total_calc:

            print()
            print('index:', i, '- BAD MATH!')
            print('   ', cand, '———', total_init, '!=', total_calc)

# drop the misspelled MATT LIERB as it is no longer needed 
for c in merged.columns:
    if 'Lierb' in c or 'Unnamed' in c: # drop the old typo column
        merged.drop(c, axis=1, inplace=True)
        continue
verify_totals(merged)

merged = merged.rename(columns={'Total':'Total Votes'})

# save


In [581]:
# convert to numeric where possible
for c in merged.columns:
    try:  merged[c] = merged[c].astype(float)
    except: pass

#### with vote types

In [582]:
merged.to_csv('all_precincts_joined/'+race_hotword+'.csv', index=False)

In [583]:
# GROUPBY FOR COUNTY DATA
num_cols = merged._get_numeric_data().columns
assert (len(num_cols)-2)%5==0
counties = merged.groupby('County').sum()
counties.to_csv('all_counties_joined/'+race_hotword+'.csv')

#### drop vote type columns

In [584]:
for col in merged.columns:
    for cand in cands:
        if cand in col and 'total' not in col.lower():
            merged.drop(col, axis=1, inplace=True)
merged.to_csv('all_precincts_joined/'+race_hotword+'_with_vote_types.csv', index=False)

In [585]:
for col in counties.columns:
    for cand in cands:
        if cand in col and 'total' not in col.lower():
            counties.drop(col, axis=1, inplace=True)
counties.to_csv('all_counties_joined/'+race_hotword+'_with_vote_types.csv')